# Training a road sign detector with tensorflow 1.x (GPU ENABLED)

I have created this notebook with the help of this article
> https://medium.com/analytics-vidhya/training-an-object-detection-model-with-tensorflow-api-using-google-colab-4f9a688d5e8b


The dataset that I will be using can be found from here 
> http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset


And the function that helped me generate the tfrecords was found from here 
> https://github.com/datitran/raccoon_dataset/blob/master/generate_tfrecord.py

In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


# Enable GPU acceleration by navigating to 

> Edit > Notebook Settings > Choose "GPU" for hardware acceleration.

Then run the cell below to make sure the GPU is setup. You should see 'Found GPU'

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')

print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# **Mount** your google drive my using **'Mount drive'** from the files section.

Change directory to the folder you created on your google drive. I used 'RoadSignDetection'. If you used a different name replace every instance of 'RoadSignDetection' to the name you have chosen.

In [0]:
%cd 'drive/My Drive/RoadSignDetection/'

/content/drive/My Drive/RoadSignDetection


Clone the Tensorflow models repository. Also you may clone the COCO repository and install the COCO object detection API for evaluation purpose.

In [0]:
!git clone https://github.com/tensorflow/models.git
%cd 'models/research'
!pip install "git+https://github.com/philferriere/cocoapi.git#egg=pycocotools&subdirectory=PythonAPI"
%cd '/content/drive/My Drive/RoadSignDetection/'

Install some needed tools.

In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

Compile the model definition. Remember to change to the name of your folder.

In [0]:
%cd '/content/drive/My Drive/RoadSignDetection/models/research/'

/content/drive/My Drive/RoadSignDetection/models/research


In [0]:
%cd '/content/drive/My Drive/RoadSignDetection/models/research/'
!protoc object_detection/protos/*.proto --python_out=.

/content/drive/My Drive/RoadSignDetection/models/research


Set the environment

In [0]:
import os
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/RoadSignDetection/models/research/:/content/drive/My Drive/RoadSignDetection/models/research/slim'

# Setting up

In [0]:
!python setup.py build
!python setup.py install

# Note: if you wish to know the remaining hours you have for your colab session run the copy and run the code below.

In [0]:
import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: ', round(Left/3600, 2), ' hours')

Time remaining for this session is:  11.89  hours


Test to see if all we need for the training has been installed.



In [0]:
!python object_detection/builders/model_builder_test.py

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTest.test_create_experimental_model
[       OK ] ModelBuilderTest.test_create_experimental_model
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[       OK ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[       OK ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_wi

# Up to here the training environment should be setup.

# Following we will insert the dataset that will be used for training, the tensorflow detection model and its .config file

In [0]:
%cd '/content/drive/My Drive/RoadSignDetection/models/research/object_detection/'

/content/drive/My Drive/RoadSignDetection/models/research/object_detection


Get the pre-trained Object detection model from tensorflow with the code below

More can be found from here:
> https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md


In [0]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
!tar -xvf ssd_mobilenet_v2_coco_2018_03_29.tar.gz

# !wget http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_2018_01_28.tar.gz
# !tar -xvf faster_rcnn_resnet101_coco_2018_01_28.tar.gz

Get its corrensponding .config file.

More can be found from here:
> https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs

In [0]:
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config

# !wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/samples/configs/faster_rcnn_resnet101_coco.config

# From here we have to do the manual part!

This is a link to my github repository <.....>

Follow the instructions from the 'Process.txt' to pre-process the downloaded road sign dataset. Come back to continue to this notebook only after you have finished pre-processing the dataset

Go to google drive, find the folder that contains everything you have been storing from this notebook. 

> Navigate to models > research > object_detection 

From here create a "dataset" folder and the insert the .record files. 

Also create a "training" folder and insert the "label_map.pbtxt" you would have created by following the instructions

Additionally, download .config model file. Open it with a text editor (such as Sublime) and configure it with the correct checkpoint path, num_classes, num_examples, and label_map_path.

In [0]:
!pwd

/content/drive/My Drive/RoadSignDetection/models/research/object_detection


# After everything has been setup we can now run the following cell where it will begin training and storing the checkpoints to the 'training' folder inside the object_detection

You can leave this running in the background. Aim to reach a loss of about 1 and then stop the execution. This is necessary for sufficient results.

While this is running you can observe the training process by opening the other notebook 'Tensorboard.ipynb' and following the steps

In the model_main.py add

tf.logging.set_verbosity(tf.logging.INFO)

after the imports and graphs, and then add

config = tf.estimator.RunConfig(model_dir=FLAGS.model_dir, log_step_count_steps=1, save_checkpoints_steps=2500)

inside the 'main' function after the 2 flags lines

In [0]:
# Combining training and evaluation
!python model_main.py \
  --pipeline_config_path=ssd_mobilenet_v2_coco.config \
  --model_dir=training \
  --alsologtostderr

  # --pipeline_config_path=faster_rcnn_resnet101_coco.config \


Streaming output truncated to the last 5000 lines.
INFO:tensorflow:loss = 1.7625389, step = 1411 (0.269 sec)
I0429 09:52:54.529496 140280750237568 basic_session_run_hooks.py:260] loss = 1.7625389, step = 1411 (0.269 sec)
INFO:tensorflow:global_step/sec: 3.67145
I0429 09:52:54.801420 140280750237568 basic_session_run_hooks.py:692] global_step/sec: 3.67145
INFO:tensorflow:loss = 1.526417, step = 1412 (0.272 sec)
I0429 09:52:54.801739 140280750237568 basic_session_run_hooks.py:260] loss = 1.526417, step = 1412 (0.272 sec)
INFO:tensorflow:global_step/sec: 4.17445
I0429 09:52:55.041034 140280750237568 basic_session_run_hooks.py:692] global_step/sec: 4.17445
INFO:tensorflow:loss = 1.8338989, step = 1413 (0.240 sec)
I0429 09:52:55.041389 140280750237568 basic_session_run_hooks.py:260] loss = 1.8338989, step = 1413 (0.240 sec)
INFO:tensorflow:global_step/sec: 4.03452
I0429 09:52:55.288828 140280750237568 basic_session_run_hooks.py:692] global_step/sec: 4.03452
INFO:tensorflow:loss = 2.1358407,

In [0]:
!pwd

# After you have trained the model for a sufficient amount of time and you stopped the execution you are able to export the inference graph which you can use for road sign detection

Go in the training folder, and find the last checkpoint saved. It should be the one with the highest number in model.ckpt-#### (but make sure all 3 files .meta / .index / .data are present) if any of them is missing take the previous checkpoint that has all 3!

Change the following cell commands to that number.

In [0]:
!python export_inference_graph.py \
--input_type image_tensor \
--pipeline_config_path faster_rcnn_resnet101_coco.config \
--trained_checkpoint_prefix training/model.ckpt-35000 \
--output_directory inference_graph_6/

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0428 19:28:56.306811 139816199374720 module_wrapper.py:139] From export_inference_graph.py:145: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0428 19:28:57.355269 139816199374720 module_wrapper.py:139] From /content/drive/My Drive/RoadSignDetection/models/research/object_detection/exporter.py:402: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0428 19:28:57.357855 139816199374720 module_wrapper.py:139] From /content/drive/My Drive/RoadSignDetection/models/research/object_detection/exporter.py:121: The name tf.placeholder is deprecated. Please use tf.comp

# By reaching the end you are now able to run road sign detection on both video and images by using the new frozen_inference_graph that will be stored on the folder you have chosen!